In [1]:
import numpy as np
import pandas as pd

import os
import pickle
import glob
import matplotlib.pyplot as plt
import time
import keras
import imageio
from scipy import signal
from keras.models import load_model
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import explained_variance_score
from sklearn.metrics import roc_curve, auc
from skimage.transform import resize
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


In [2]:
save_figures = False
file_ending = '.png'
model_string = 'NMDA'

dataset_folder = '../data'

models_folder     = os.path.join(dataset_folder, 'Models')
morphology_folder = os.path.join(dataset_folder, 'Morphology')
test_data_folder  = os.path.join(dataset_folder, 'Data_test')
#auxiliary_folder  = os.path.join(dataset_folder, 'Auxiliary')

model_filename           = os.path.join(models_folder, 'NMDA_TCN__DWT_7_128_153__model.h5')
model_metadata_filename  = os.path.join(models_folder, 'NMDA_TCN__DWT_7_128_153__training.pickle')
morphology_filename      = os.path.join(morphology_folder, 'morphology_dict.pickle')
#NN_illustration_filename = os.path.join(auxiliary_folder, 'TCN_7_layers.png')
test_files               = sorted(glob.glob(os.path.join(test_data_folder, '*_128_simulationRuns*_6_secDuration_*')))

print('-----------------------------------------------')
print('finding files: morphology and test data')
print('-----------------------------------------------')
#print('model found          : "%s"' %(model_filename.split('/')[-1]))
print('model metadata found : "%s"' %(model_metadata_filename.split('/')[-1]))
print('morphology found     : "%s"' %(morphology_filename.split('/')[-1]))
print('number of test files is %d' %(len(test_files)))
print('-----------------------------------------------')

-----------------------------------------------
finding files: morphology and test data
-----------------------------------------------
model metadata found : "NMDA_TCN__DWT_7_128_153__training.pickle"
morphology found     : "morphology_dict.pickle"
number of test files is 12
-----------------------------------------------


In [3]:
experiment_dict = pickle.load(open(test_files[0], "rb" ), encoding='latin1')


In [4]:
experiment_dict.keys()
df = pd.DataFrame(experiment_dict)

In [28]:
df

,Params,Results
useActiveDendrites,True,NaN
randomSeed,100520,NaN
collectAndSaveDVTs,True,NaN
numOutputSpikesPerSim,"[15, 0, 2, 0, 7, 30, 0, 10, 1, 20, 13, 17, 3, ...",NaN
averageOutputFrequency,0.730469,NaN
numSimulations,128,NaN
totalSimDurationInSec,6,NaN
num_bas_ex_spikes_per_100ms_range,"[0, 750]",NaN
totalNumSimulationSeconds,768,NaN
allSegmentsLength,"[24.200878230108255, 15.18219623504249, 21.664...",NaN


In [17]:
experiment_metadata_dict['data_dict'].keys()

dict_keys(['train_files', 'test_files', 'valid_files'])

In [ ]:
##%% helper functions

def bin2dict(bin_spikes_matrix):
    spike_row_inds, spike_times = np.nonzero(bin_spikes_matrix)
    row_inds_spike_times_map = {}
    for row_ind, syn_time in zip(spike_row_inds,spike_times):
        if row_ind in row_inds_spike_times_map.keys():
            row_inds_spike_times_map[row_ind].append(syn_time)
        else:
            row_inds_spike_times_map[row_ind] = [syn_time]

    return row_inds_spike_times_map


def dict2bin(row_inds_spike_times_map, num_segments, sim_duration_ms):
    
    bin_spikes_matrix = np.zeros((num_segments, sim_duration_ms), dtype='bool')
    for row_ind in row_inds_spike_times_map.keys():
        for spike_time in row_inds_spike_times_map[row_ind]:
            bin_spikes_matrix[row_ind,spike_time] = 1.0
    
    return bin_spikes_matrix

In [7]:
'''experiment_dict = pickle.load(open(test_files[0], "rb" ), encoding='latin1')

# gather params
num_simulations = len(experiment_dict['Results']['listOfSingleSimulationDicts'])
num_segments_basal    = len(experiment_dict[experiment_dict['Params']['allSegmentsType'] == 'basal'])
num_segments_apical    = len(experiment_dict[experiment_dict['Params']['allSegmentsType'] == 'apical'])
sim_duration_ms = experiment_dict['Params']['totalSimDurationInSec'] * 1000
num_ex_synapses_b  = num_segments_basal /2
num_inh_synapses_b = num_segments_basal/2 
num_ex_synapses_a  = num_segments_apical/2 
num_inh_synapses_a = num_segments_apical/2
num_synapses_b = num_ex_synapses_b + num_inh_synapses_b
num_synapses_a = num_ex_synapses_a + num_inh_synapses_a''';

KeyError: False